In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

#Reading the dataset in a dataframe using Pandas
df = _____

#Let's look at the data!
df.head(10)

In [ ]:
#Now let's learn about some of the numerical fields
_____

In [ ]:
#Another way to understand the numerical data is to visualize it
#We can plot one of these distributions easily in just one line
df['ApplicantIncome'].hist(bins=50)

In [ ]:
#We can also use box plots and segment the information out
df.boxplot(column='ApplicantIncome', by = 'Education')

In [ ]:
#To understand categorical (non-numerical) features we can look at the distribution to gain a better understanding 
______

In [ ]:
'''
Now let's get into part of what we are here for, predicting loan approval
To me, one of the variables that could be important is whether you have a credit history
We'll start by looking at the distribution of that field
'''

temp1 = df['Credit_History'].value_counts(ascending=True)
temp2 = df.pivot_table(values='Loan_Status',index=['Credit_History'],aggfunc=lambda x: x.map({'Y':1,'N':0}).mean())
print ('Frequency Table for Credit History:') 
print (temp1)

print ('\nProbility of getting loan for each Credit History class:')
print (temp2)

In [ ]:
#As to be expected, it is much easier to get a loan once you have at least some credit history
#This is ploted so it is easier to see below

temp3 = _____
temp3.plot(kind='bar', stacked=True, color=['red','blue'], grid=False)

In [ ]:
#You can also add gender into the mix and start creating a very basic classification algorithm 
#These are similar to pivot tables in excel, just with more functionality and can process more data
temp4 = pd.crosstab([df.Credit_History, df.Gender], df['Loan_Status'])
print(temp4)
temp4.plot(kind='bar', stacked=True, color=['red','blue'], grid=False)

In [ ]:
'''
Now before we move along to build actual models, we must go back to some of the initial problems we noticed
when we first reviewed the data. This included:
1) Missing values
2) Extremes/Outliers

Topic (2) is sometthing we could look into given more time, but missing values are a much bigger issue
and so I will only focus on (1) during this workshop
'''

#Use the function below to check the number of null (empty) values for each field
df.apply(lambda x: sum(x.isnull()),axis=0) 


In [ ]:
'''
How might we address some of these errors?
One way would be to get rid of that feature in our analysis entirely. 
- If this field is relevant, that may not be the best
Another way would be to set it to the mean, median, or mode. We'll use the mean value below
'''
#Commented out because we will discuss a better way later
#df['LoanAmount'].fillna(df['LoanAmount'].mean(), inplace=True)

In [ ]:
#For categorical values it may be more difficult. In the case of binary variables we can choose the more common case
_____

In [ ]:
df['Self_Employed'].fillna('No',inplace=True)

In [ ]:
'''
A more complicated way to fill in the null loan amount values would be to break find the mean/median/mode
for each segment. In the code below, we break this up by the binary "Self_Employed" and "Education" features
and then use the median value for each category, somewhat factoring out the extremes
'''

table = df.pivot_table(values='LoanAmount', index='Self_Employed' ,columns='Education', aggfunc=np.median)
# Define function to return value of this pivot_table
def fage(x):
 return table.loc[x['Self_Employed'],x['Education']]
# Replace missing values
df['LoanAmount'].fillna(df[df['LoanAmount'].isnull()].apply(fage, axis=1), inplace=True)

In [ ]:
#Another way to factor in extremes is to use logs of the input values.
#We see a much nicer distribution of Loan Amounts when we use that below

df['LoanAmount_log'] = _____
df['LoanAmount_log'].hist(bins=20)

In [ ]:
#Now let's finish out addressing the null values for the remaining fields:
#Gender, Married, Dependents, Loan_Amount_Term, and Credit_History

df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
df['Married'].fillna(df['Married'].mode()[0], inplace=True)
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)

#Since credit history is such a deciding factor with many missing values, I'm going to drop these cases altogether
df.dropna(axis=0, subset=['Credit_History'], inplace=True)

#Clearly there are additional (probably better) ways to adjust dealing with these missing values, but given the
#timeframe of the workshop we won't bother with dig into other options right now.


In [ ]:
#Check that the null values are actually gone (it didn't work at first when I was going through this workshop)
df.head(100)

In [ ]:
#You can use the commented out code below to check the data types of each of the columns in your dataframe
#df.dtypes

In [ ]:
#Now before we actually test, make sure to seperate the labels from the features
# Filter to only the features
train_predictors = df.loc[:, df.columns != 'Loan_Status']
# The target we're using to train the algorithm
train_target = df['Loan_Status']


In [ ]:
#We are so close to being able to build a model! 
#We'll begin the preparation by breaking the training data into training and validation data
# (This is typically done when you have a medium-sized data set, and isn't as important here with only 624 samples)

from sklearn.model_selection import train_test_split
#Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(train_predictors, train_target, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

#"Cardinality" means the number of unique values in a column - found in the nunique() function
#Select categorical columns with relatively low cardinality (convenient but arbitrary)
#Again, in our case this isn't as relevant but it would totally be for claims data
categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]

#Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

#Keep selected columns only
my_cols = categorical_cols + numerical_cols
train_df = X_train[my_cols].copy()
valid_df = X_valid[my_cols].copy()


In [ ]:
#We'll import a bunch of librarys we will use for a few steps of pre-processing, and then building models

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Let's pre-process the data to one-hot encode the categorical variables before building the model
#Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

#Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
#Now we can build our first model!!!

#We'll use XGBoost classifier

#Define the model
XGB_model = ______

#Bundle preprocessing and modeling code in a pipeline
classifier = Pipeline(steps=[('preprocessor', _____),
                      ('model', _____)
                     ])

#Preprocessing of training data, fit model 
classifier.fit(_____, _____)

#Preprocessing of validation data, get predictions
easypreds = classifier.predict(X_train)
preds = classifier.predict(X_valid)

accuracy = accuracy_score(y_train, easypreds)
cv_accuracy = accuracy_score(y_valid, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Cross-Validation Accuracy: %.2f%%" % (cv_accuracy * 100.0))

In [ ]:
'''
Now that is probably not as great it as we'd like it to be, but what we can do is change the parameters

The default parameters are given below:

max_depth=3
learning_rate=0.1
n_estimators=100
silent=True
objective='binary:logistic'
booster='gbtree'
n_jobs=1
nthread=None
gamma=0
min_child_weight=1
max_delta_step=0
subsample=1
colsample_bytree=1
colsample_bylevel=1
reg_alpha=0
reg_lambda=1
scale_pos_weight=1
base_score=0.5
random_state=0
seed=None
missing=None
'''

In [ ]:
#We probably want a greater tree depth
#Let's try it again below

#Define model
XGB_model2 = XGBClassifier(max_depth=_____)

#Bundle preprocessing and modeling code in a pipeline
classifier2 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', XGB_model2)
                     ])

#Preprocessing of training data, fit model 
classifier2.fit(X_train, y_train)

#Preprocessing of validation data, get predictions
easypreds = classifier2.predict(X_train)
preds = classifier2.predict(X_valid)

accuracy = accuracy_score(y_train, easypreds)
cv_accuracy = accuracy_score(y_valid, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Cross-Validation Accuracy: %.2f%%" % (cv_accuracy * 100.0))
